In [4]:
import pandas

# import the data
df = pandas.read_csv('../data/tcga_brca_all_clean.csv')

df

,case_id,slide_id,site,is_female,oncotree_code,age,survival_months,censorship,train,NOTCH2_cnv,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
0,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,3C,1.0,IDC,50.0,131.57,1.0,1.0,-1,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
1,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF...,3C,1.0,IDC,50.0,131.57,1.0,1.0,-1,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
2,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX1.777C0957-255A-42F0-9EE...,3C,1.0,IDC,62.0,48.42,1.0,1.0,0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
3,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-962...,3C,1.0,IDC,62.0,48.42,1.0,1.0,0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
4,TCGA-3C-AALK,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,3C,1.0,IDC,52.0,47.57,1.0,1.0,0,...,-0.2431,-0.3657,-0.4472,-1.0557,0.8232,-1.0342,1.0729,-1.4634,-0.5710,-0.5402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,TCGA-WT-AB44,TCGA-WT-AB44-01Z-00-DX1.B6ECEA7C-DA26-4B34-88C...,WT,1.0,ILC,77.0,29.01,1.0,1.0,2,...,-1.1686,-1.5424,-1.0868,-1.4608,-0.0141,-2.0773,3.9959,-1.4221,-1.0560,1.7134
1019,TCGA-XX-A899,TCGA-XX-A899-01Z-00-DX1.08FE27B7-73B8-4CE3-ACF...,XX,1.0,ILC,46.0,15.34,1.0,1.0,0,...,-0.6444,0.2435,0.2311,0.7384,-0.8560,-0.7456,0.7535,0.9765,-0.6290,-0.4313
1020,TCGA-XX-A89A,TCGA-XX-A89A-01Z-00-DX1.671E2AD6-4D1A-4579-88C...,XX,1.0,ILC,68.0,16.03,1.0,1.0,0,...,-0.1942,-0.6109,-0.7641,1.2241,0.7029,-0.5363,0.9538,-0.0743,-0.0954,0.0371
1021,TCGA-Z7-A8R5,TCGA-Z7-A8R5-01Z-00-DX1.3BDB407F-514C-4131-B05...,Z7,1.0,ILC,61.0,107.98,1.0,1.0,2,...,-1.0210,-1.2416,-1.0705,-1.3571,-0.5215,-1.3964,1.6214,-1.5199,-0.4771,0.0497


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Select numeric columns and preprocess
numeric_columns = [col for col in df.columns if df[col].dtype in [np.int64, np.float64]]
df_numeric = df[numeric_columns]

# Define risk groups based on median survival months
median_survival = df_numeric['survival_months'].median()
df_numeric['risk_group'] = (df_numeric['survival_months'] < median_survival).astype(int)

# Drop unnecessary columns
df_numeric = df_numeric.drop(columns=['survival_months', 'censorship', 'train'])

# Extract features and labels
X = df_numeric.drop(columns=['risk_group']).values
y = df_numeric['risk_group'].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for CNN input
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42, stratify=y)

# Define CNN model
model = keras.Sequential([
    layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_reshaped.shape[1], 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=16,
    verbose=1
)


/tmp/ipykernel_1661349/2884203551.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['risk_group'] = (df_numeric['survival_months'] < median_survival).astype(int)
/home/yliu802/miniconda3/envs/csci2470/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-11 11:10:48.301054: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 13s 177ms/step - accuracy: 0.5204 - loss: 17.4066 - val_accuracy: 0.5659 - val_loss: 2.5009
Epoch 2/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 172ms/step - accuracy: 0.6872 - loss: 0.5816 - val_accuracy: 0.5805 - val_loss: 5.5911
Epoch 3/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 174ms/step - accuracy: 0.7130 - loss: 0.4970 - val_accuracy: 0.5854 - val_loss: 6.4380
Epoch 4/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 169ms/step - accuracy: 0.7971 - loss: 0.3825 - val_accuracy: 0.5902 - val_loss: 6.6245
Epoch 5/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 167ms/step - accuracy: 0.8298 - loss: 0.2744 - val_accuracy: 0.5756 - val_loss: 5.6165
Epoch 6/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 165ms/step - accuracy: 0.8845 - loss: 0.2409 - val_accuracy: 0.5951 - val_loss: 3.7242
Epoch 7/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 166ms/step - accuracy: 0.8338 - loss: 0.2647 - val_accuracy: 0.6098 - val_loss: 1.8563
Epoch 8/30
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 167ms/step - accuracy: 0.8575 - loss: 0.2160 - val_accuracy: 

In [6]:

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6851 - loss: 1.0518
Test Accuracy: 0.6439
